In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

In [53]:
# The location data set of the sensors.
df_s = pd.read_csv('https://www.student.dtu.dk/~s212220/socialData2022/Data/Pedestrian_Counting_System_-_Sensor_Locations.csv')

In [3]:
# The pedestrian data set
df_p = pd.read_csv('https://www.student.dtu.dk/~s212220/socialData2022/Data/Pedestrian_Counting_System_-_Monthly__counts_per_hour_.csv')

In [55]:
# The weather data set
df_w = pd.read_csv('https://www.student.dtu.dk/~s212220/socialData2022/Data/weather_melbourne.csv')

### A short section on the dataset stats, containing key points/plots from our exploratory data analysis.

Let's start with the location of the sensors. Here are some basic information about it:
* 78 rows coresponding to the amount of sensors.
* 10KB
* 11 columns: only bold written columns will be used.
    * **sensor_id**: integers from 1 to 79 correcponting to each row and sensor
    * **sensor_description**: unique categories (78 values) which corespond to the street name the sensors are installed on
    * sensor_name: categorial unique name (78 values) of the senor
    * **installation_date**: dates when the sensor was installed, 60 different values from 2009/01/20 until 2021/09/10
    * **status**: categorial (A = Active, I = Inactive, R = Removed), 71 sensors are active and 7 where removed
    * note: categorial strings, notes on a sensor, 10 different values, the other 69 values are missing
    * direction_1 & direction_2: categorial (North, South, West, East) the two directions the sensors are reading into. Both have 4 missing values
    * **latitude**: distinct real numbers representing the latitude of the sensor, from -37.82401776 until -37.79453802
    * **longitude**: distinct real numbers representing the longitude of the sensor, from 144.9303619 until 144.9746766
    * location: 78 distinct values showing the latitude and longitude together.

An imporant thing to notice is the status.  7 sensors were removed in the past due to construction work or something else. Since we assume that there were no futher data collected from the sensors after their removal, we can still leave them in the dataset. We always have to be aware that the amount of sensors collecting data changes over time. Either to new installed sensors or to removed ones. (While looking at the data points of the removed sensors, it states that 2 of them were removed in 2017, one in 2015 and for the other 4 is no information provided).

In [6]:
df_s.drop(['sensor_name', 'note', 'direction_1', 'direction_2', 'location'], axis=1, inplace=True)

In [7]:
# The maps shows the location of the sensors.
map_loc = folium.Map(location=[-37.8140000, 144.9633200],tiles = "Stamen Toner",
                    zoom_start = 14) 

# Ensure you're handing it floats
df_s['latitude'] = df_s['latitude'].astype(float)
df_s['longitude'] = df_s['longitude'].astype(float)

for index, row in df_s.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['sensor_description']).add_to(map_loc)

# Styling title
loc = 'Corpus Christi'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>Locations of Sensors</b></h3>
             '''.format(loc) 
map_loc.get_root().html.add_child(folium.Element(title_html))

# Display the map
map_loc

NameError: name 'folium' is not defined

The interactive map shows all the locations of the different sensors collecting the pedestrian data. As visible, most of the sensors are located in the inner city and that's where our focus is directed in the project. Each of the sensors provides a description containing the street or corner where they are located. One outlineer can be noticed when zooming out on the map. It's the sensor located on the corner of Macaulay Rd and Bellair St. As we are focusing on the city center and - by having a look at the data - since it was installed at the end of the timeframe of our pedestrian data set on 2021/02/20, we still leave it in the data set but our later maps may not focus on showing it as well.

In [ ]:
# finding the year of the installation
df_s['Year'] = pd.DatetimeIndex(df_s['installation_date']).year
# adding missing years in range(2009, 2022)
df = pd.DataFrame({'Year': [2010, 2011, 2012, 2016]})
df_s_y = df_s.append(df, ignore_index = True)

# Plot that shows new introduced sensors per year
fig = plt.figure(figsize=(5, 4))
df_s_y.groupby(['Year']).count()['sensor_id'].plot \
    .bar(title='New Installed Sensors per Year', x=range(df_s['Year'].min(),df_s['Year'].max() + 1),
         edgecolor='darkgreen', color = 'green')
# Styling
plt.ylabel('Number of New Installed Sensors');

plt.tight_layout() 
plt.show()

The plot aboth shows when sensors were installed over the years. Covid is a pendemic which hit Melbourne in March 2020 [Source](https://en.wikipedia.org/wiki/COVID-19_pandemic_in_Victoria). Until then there were 61 sensors installed. In the futher analytics we will focus on 2019 as a year before covid and 2020 as a representativ of covid.

Next up, we are providing some important insights to the pedestrian data we used:
* 4176218 rows corresponding to data collected in an hour with one sensor.
* 396.8MB
* 10 columns: only bold written columns will be used.
    * ID: distinct number representing a unique id of the data point from 1 to 4181689
    * Date_Time: categorial string value representing the date and time the data point was collected
    * **Year**: 16 different number values representing the year when the data point was collected, from 2009 until 2022
    * **Month**: 12 categorial string values representing the month of the year.
    * **Mdate**: 31 integers from 1 until 31 representing the date of the month.
    * **Day**: 7 categorial string values representing the day of the week.
    * **Time**: 23 integers representing the starting time of the day the data was collected for an hour.
    * **Sensor_ID**: 81 different integers representing the ID of the sensor collecting the data. Reference to the sensor location data set.
    * Sensor_Name: categorial unique name (93 values) of the senor collecting the data
    * **Hourly_Counts**: 6359 different numbers representing the total amount of pedestrians counted during this hour form 0 to 15979

In [ ]:
# Dropping unused columns
df_p.drop(['ID', 'Date_Time', 'Sensor_Name'], axis=1, inplace=True)

With this information, we can see that there is a difference in amount of Sensor_ID and Sensor_Name which should not happen. Since it is only the name which probably relates to a different naming and we do not need this column, we are not looking deeper into that. 

More interesting is the difference between the sensor ids of the pedestrian dataset and the sensor location data set. Let's have a look at that:

In [ ]:
# Finding the 3 missing sensors
for s_id in df_p.Sensor_ID.unique():
    if s_id not in df_s.sensor_id.unique():
        print('Sensor ID: ', s_id)

A we can see, the sensor ids 84, 85 and 86 are not appearing in the sensor location data set. We decided to drop the data points collected with these sencors since we are not having enouth information about the senors.

In [5]:
# Cleaning the pedestrian data - deleting the missing sensors activity
df_p = df_p[(df_p.Sensor_ID != 84) & (df_p.Sensor_ID != 85) & (df_p.Sensor_ID != 86)]

In [ ]:
# Plot shows the data points over the year and the amount of pedestrians walking
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,4))
df_p.groupby('Year').count()['ID'].plot \
    .bar(title='Data points over the Years\n', 
         ax = ax[0], edgecolor='darkgreen', color = 'green')
df_p.groupby(['Year']).sum()['Hourly_Counts'].plot \
    .bar(title='Amount of Pedestrians on the Street per Year\n', 
         ax=ax[1], edgecolor='darkgreen', color = 'green')

# Styling
ax[0].set_ylabel('Num of Data Points');
ax[1].set_ylabel('Total Amount of Pedestrians');

plt.tight_layout() 
plt.show()

The collection of data grew over the years for the pedestrian data set. This can be compared with the plot aboth showing the installed sensors. Whenever no new sensors where installed, the collection of data languishs. The amount of pedestrians on the street is mainly growing due to more datapoints but we can see a few unexpected outliners. Looking at 2020 and 2021, we can see a huge drop in the amount of pedestrians on the street while the data points are growing which is due to covid. 
Another small drop can be observed in 2017 which is probably due to the sensors which were removed during that year.

Finally, we are digging deeper into the weather data set:
* 2557 rows corresponding to each day
* 114KB
* 11 columns: only bold written columns will be used.
    * **date**: distinct categorial value representing the date the data point was collected form 2015/01/01 until 2021/12/31
    * **tavg**: 251 different real numbers representing the average temperature on that day from 4.6 to 34.3 °C
    * **tmin**: 230 different real numbers representing the minimum temperature on that day from -2 to 26.5 °C
    * **tmax**: 305 different real numbers representing the maximum temperature on that day from 8.3 to 46 °C
    * **prcp**: 120 different real numbers representing the percipitation on that day from 0 to 69 mm
    * snow: completly missing
    * **wdir**: 277 different real numbers representing the wind direction on that day from 0 to 360 degrees, 160 missing values
    * **wspd**: 343 different real numbers representing the wind speed on that day from 3.8 to 55.3 km/h, 21 missing values
    * wpgt: comletly missing
    * pres: 358 different real numbers representing the air pressure on that day from 962.2 to 1036.1 hPa, 350 missing values
    * tsun: completly missing
    
Due to either comletly missing all values or not enouth values (14% missing), we decided to drop the columns which are not written in bold. Since we still have some missing values, we will drop the rows having missing values.

In [61]:
# Cleaning the weather data - dropping snow, wpgt, tsun, pres due to too many missing values
df_w = df_w.drop(['snow', 'wpgt', 'pres', 'tsun'], axis=1)
# Drop NaN values of the rest of the culomns
df_w = df_w.dropna()

'2021-12-31'

In [ ]:
months = ["January", "February", "March", "April", "May", "June", 
          "July", "August", "September", "October", "November", "December"]
# Prefparing the weather data
df_w['Year'] = pd.DatetimeIndex(df_w['date']).year
df_w['MonthNo'] = pd.DatetimeIndex(df_w['date']).month
df_w['Month'] = df_w['MonthNo'].apply(lambda x: months[x-1])

# Creating plots with basic statistics for temperature, wind-speed and percipitation
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(13,7))
df_w['Month'] = pd.Categorical(df_w['Month'], categories=months, ordered=True)

featuresToPlot = ['prcp','wspd','tavg']
titleLabels = ['precipitation','wind-speed','temperature']
colors = ['blue','grey','red']
yUnits = ['mm','km / h','C°']
for n, f in enumerate(featuresToPlot):
    df_w.groupby(df_w['Year']).mean()[f].plot.bar(color = colors[n],title=f'Average {titleLabels[n]} over years', ax = ax[(int(n/3)),(int(n%3))])
    ax[(int(n/3)),(int(n%3))].set_ylabel(yUnits[n]) 
    
for n, f in enumerate(featuresToPlot):
    df_w.groupby(df_w['Month']).mean()[f].plot.bar(color = colors[n],title=f'Average {titleLabels[n]} over months', ax = ax[(int((n+3)/3)),(int((n+3)%3))])
    ax[(int((n+3)/3)),(int((n+3)%3))].set_ylabel(yUnits[n]) 

plt.tight_layout() 
plt.show()

By looking at a few basic plots of the weather data, we can see that 2015 and 2019 where quite dry years. Looking at the rain over months, it reveals a rather unusual structure since there are some hugh drops. We din't expect that but we think its just how weather is. Not always as expected. The other plots show that Melbourne has quite static weather weather over the years with a normal distribution over the months.